<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2025notebooks/2025_1017olivetti_face_LSTM_SRN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# オリベッティ顔データセットを用いた機械学習実習

* filename: 2025_1017olivetti_face_LSTM_SRN.ipynb
* author: 浅川 伸一 asakawa@ieee.org
* date: 2025_1010




In [ ]:
%config InlineBackend.figure_format = 'retina'
import IPython
isColab = 'google.colab' in str(IPython.get_ipython())

import matplotlib.pyplot as plt
try:
    import japanize_matplotlib
except ImportError:
    !pip install japanize_matplotlib
    import japanize_matplotlib

import numpy as np
# numpy の表示桁設定
np.set_printoptions(precision=5, suppress=False)

# オリベッティ顔データセットの読み込み
from sklearn.datasets import fetch_olivetti_faces
data = fetch_olivetti_faces()
X, y = data.data, data.target
print(f'訓練データサイズ X.shape:{X.shape}')
print(f'教師データサイズ y.shape:{y.shape}')
print(data.DESCR)

In [ ]:
# データの表示
nrows = 40   # nrows 人分のデータを表示
ncols = 10
fig, fig_axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(ncols * 1.4, nrows * 1.4), constrained_layout=True)
# constrained_layout は subplot や 凡例やカラーバーなどの装飾を自動的に調整して，
# ユーザが要求する論理的なレイアウトをできるだけ維持しながら， 図ウィンドウに収まるようにします。

for i in range(nrows):
    for j in range(ncols):
        x = i * 10 + j
        fig_axes[i][j].imshow(X[x].reshape(64,64), cmap='gray')
        fig_axes[i][j].axis('off')
        fig_axes[i][j].set_title(f'num:{x}, ID:{y[x]}')

plt.show()

In [ ]:
# 男女の判別のため教師データを作成
# 男であれば 0, 女であれば 1 とする
y_sex = np.zeros_like(y)
for woman_start in [70, 90, 310, 340]:
    for i in range(10):
        y_sex[woman_start+i]=1

for i in range(0, len(X),10):
    print(f'{i:3d}', end=":")
    for j in range(10):
        print(y_sex[i+j], end=" ")
    print("")

# 系列情報処理実習

データである 400 枚の画像はそれぞれ縦横 64 画素である。これを 1 次元にすれば 1 枚の画像は 4096 次元のベクトルである。
したがって，系列予測課題とみなすこともできる。

以下ではデータを一次元系列データとして視覚化を行う


In [ ]:
# 表示させたい画像番号を指定するリスト
image_nums = [0, 10, 20, 30]
image_nums = [70, 90, 310, 340]

figs, axes = plt.subplots(len(image_nums), 2, figsize=(12, 1.5 * len(image_nums)), tight_layout=False)
for i, image_num in enumerate(image_nums):
    x = X[image_num]

    #figs, axes = plt.subplots(1,2, figsize=(12,3), tight_layout=False)
    #figs.suptitle(f'データ番号:{N}')

    axes[i][0].plot(x)
    axes[i][0].set_ylim(0,1)

    axes[i][1].imshow(x.reshape(64,64), cmap='gray')
    axes[i][1].set_xticks([])
    axes[i][1].set_yticks([])

plt.tight_layout()
plt.show()

# SRN, LSTM リカレントニューラルネットワークモデルによる近似

In [6]:
# 系列データを PyTorch で使えるような形式の系列データに変換
import torch

class _dataset(torch.utils.data.Dataset):
    def __init__(self, X:np.array=X, y:np.array=y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx:int):
        inp = torch.Tensor(self.X[idx].reshape(64,64))
        tch = self.y[idx]
        return inp, tch

_ds = _dataset(X=X, y=y)       # こちらは人物同定
#_ds = _dataset(X=X, y=y_sex)  # 性別判定

# データを訓練データと検証データとに分割
N_train = int(_ds.__len__() * 0.9)  # 訓練データ数
N_val = _ds.__len__() - N_train     # 検証データ数 = 全データ - 訓練データ

# データ分割
train_ds, val_ds = torch.utils.data.random_split(
    dataset=_ds,
    lengths=(N_train,N_val), generator=torch.Generator().manual_seed(42))

# 上で分割したデータをデータローダに変換
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True)
val_dl   = torch.utils.data.DataLoader(val_ds,   batch_size=32, shuffle=False)

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim

class _SRN(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim # 中間層の次元数
        self.layer_dim = layer_dim   # 中間層の層数
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='tanh')
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        hid = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        out, hid = self.rnn(x, hid)
        out = self.fc(out[:, -1, :])
        return out


class _LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim # 中間層の次元数
        self.layer_dim = layer_dim   # 中間層の層数
        self.rnn = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):

        # Initialize hidden state with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        h1 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        hid = (h0, h1)
        out, hn = self.rnn(x, hid)
        out = self.fc(out[:, -1, :])
        return out

input_dim = 64    # 入力層の次元数
hidden_dim = 256  # 中間層の次元数
layer_dim = 2     # 中間層の層数
output_dim = 40    # 出力層の次元数
#output_dim = 2    # 出力層の次元数

model0 = _SRN(input_dim, hidden_dim, layer_dim, output_dim)
model1 = _LSTM(input_dim, hidden_dim, layer_dim, output_dim)
error = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer0 = torch.optim.Adam(model0.parameters(), lr=learning_rate)
optimizer1 = torch.optim.Adam(model1.parameters(), lr=learning_rate)

In [ ]:
seq_dim = 64
num_epochs = 50

model = model1
optimizer = optimizer1
# model = model0
# optimizer = optimizer0

loss_list = []
iteration_list = []
accuracy_list = []
count = 0
for epoch in range(num_epochs):

    losses = 0
    total = 0
    correct = 0
    model.eval()
    for inps, tchs in val_dl:
        outs = model(inps)
        preds = torch.max(outs.data, 1)[1] # Get predictions from the maximum value
        total += tchs.size(0)
        correct += (preds == tchs).sum()

    accuracy = 100 * correct / float(total)
    iteration_list.append(epoch+1)

    model.train()
    for i, (inps, tchs) in enumerate(train_dl):
        optimizer.zero_grad()
        outs = model(inps)
        loss = error(outs, tchs)
        loss.backward() # Calculating gradients
        optimizer.step() # Update parameters
        losses += loss.item()

    print(f'epoch:{epoch:03d}  Loss: {losses:8.3f}  Accuracy: {accuracy:6.3f}%')
    accuracy_list.append(accuracy)
    loss_list.append(losses) # store loss and iteration

In [ ]:
# visualization loss
plt.plot(iteration_list,loss_list)
plt.xlabel("Number of iteration")
plt.ylabel("Loss")
plt.title("RNN: Loss vs Number of iteration")
plt.show()

# visualization accuracy
plt.plot(iteration_list,accuracy_list,color = "red")
plt.xlabel("Number of iteration")
plt.ylabel("Accuracy")
plt.title("RNN: Accuracy vs Number of iteration")
#plt.savefig('graph.png')
plt.show()

In [ ]:
# リカレントニューラルネットワークモデルの定義
class _SRN(nn.Module):
    """単純再帰型ニューラルネットワーク a.k.a. エルマンネット"""
    def __init__(self, input_size=1, hidden_size=16, num_layers=1):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.rnn(x)         # RNN output for all time steps
        out = out[:, -1, :]          # Take output from the last time step
        return self.fc(out)          # Pass through linear layer


class _LSTM(nn.Module):
    """長‐短期記憶 LSTM: Long-Short Term Memory ネットワーク"""
    def __init__(self, input_size=1, hidden_size=64, num_layers=1):
        super().__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.rnn(x)         # RNN output for all time steps
        out = out[:, -1, :]          # Take output from the last time step
        return self.fc(out)          # Pass through linear layer

In [ ]:
# 系列データを PyTorch で使えるような形式の系列データに変換
class seq_dataset(torch.utils.data.Dataset):
    def __init__(self, data:np.array, seq_len:int=10):
        self.data = data
        self.seq_len = seq_len
        _X, _y = self.make_sequences(data)
        self._X = _X
        self._y = _y

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx:int):
        inp = torch.Tensor(self._X[idx]).reshape(-1,1)
        tch = torch.Tensor([self._y[idx]])
        return inp, tch

    def make_sequences(self, data:np.array):
        xs, ys = [], []
        for i in range(len(data) - self.seq_len):
            x = data[i:i+self.seq_len] # Sequence of length `seq_length`
            y = data[i+self.seq_len]   # Target is the next value
            xs.append(x)
            ys.append(y)
        return np.array(xs), np.array(ys)

## データの選択

In [ ]:
# 使用するデータを選ぶ
N = 70  # N の値は 0 から 399 までの整数，オリベッティ顔画像データの刺激番号
data = X[N]
s_ds = seq_dataset(data=data)

# データを訓練データと検証データとに分割
N_train = int(s_ds.__len__() * 0.9)  # 訓練データ数
N_val = s_ds.__len__() - N_train     # 検証データ数 = 全データ - 訓練データ

# データ分割
train_ds, val_ds = torch.utils.data.random_split(
    dataset=s_ds,
    lengths=(N_train,N_val), generator=torch.Generator().manual_seed(42))

# 上で分割したデータをデータローダに変換
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True)
val_dl   = torch.utils.data.DataLoader(val_ds,   batch_size=32, shuffle=False)

#

## モデルの実体化と訓練の実施:

以下のセル 2 行目の hidden_size を変化させて性能を比較せよ

In [ ]:
# モデルを実体化
hidden_size = 8  # 中間層の素子数
model0 = _SRN(hidden_size=hidden_size);model0.eval()
model1 = _LSTM(hidden_size=hidden_size);model1.eval()

# 損失関数の定義: MSE 平均二乗誤差 Mean Square Error
criterion = nn.MSELoss()

# 学習に用いる最適化関数の定義
lr = 1e-2  # lr: learning rate
optimizer0 = optim.Adam(model0.parameters(), lr=lr)
optimizer1 = optim.Adam(model1.parameters(), lr=lr)

# 実習用，上で定義した SRN か LSTM かを選択する
_model = model0
_optimizer = optimizer0


EPOCHS = 30                        # 訓練回数を設定
train_losses, val_losses = [], []  # 訓練損失値と検証損失値を保存するための空リストを定義
for epoch in range(EPOCHS):

    _model.eval()
    losses = []
    for inps, tchs in val_dl:
        outs = _model(inps)
        loss = criterion(outs, tchs)
        losses.append(loss.item())
    val_losses.append(np.mean(losses))

    _model.train()
    losses = []
    for inps, tchs in train_dl:
        outs = _model(inps)
        loss = criterion(outs, tchs)

        _optimizer.zero_grad()
        loss.backward()
        _optimizer.step()
        losses.append(loss.item())
    train_losses.append(np.mean(losses))

    if (epoch + 1) % 1 == 0:
        print(f"Epoch [{epoch+1:03d}/{EPOCHS:03d}]",
              f'訓練損失:{train_losses[-1]:8.4f}',
              f'検証損失:{val_losses[-1]:8.4f}' )

# 学習曲線を描画
plt.figure(figsize=(6,3))
plt.plot(train_losses, label='訓練誤差')
plt.plot(val_losses, label='検証誤差')
plt.show()

In [ ]:
# 画像の復元
_model.eval()
y_hats, ys = [], []
for i in range(s_ds.__len__()):
    x, y = s_ds.__getitem__(i)
    y_hat = _model(x.unsqueeze(0))
    y_hats.append(y_hat.cpu().detach().numpy()[0])
    ys.append(y.cpu().detach().numpy()[0])

y_hats, ys = np.array(y_hats), np.array(ys)

In [ ]:
from matplotlib import gridspec

fig = plt.figure(figsize=(12, 2)) # 全体の図の大きさを指定
gs = gridspec.GridSpec(ncols=3, nrows=1,
                       width_ratios=[4, 1,1],
                       #height_ratios=[1, 2],
                       wspace=0.1) # , hspace=0.4) # Adjust spacing between subplots

ax0 = fig.add_subplot(gs[0, 0]) # left
ax0.plot(range(2000), y_hats[-2000:], color='red', label='予測')
ax0.plot(range(2000), ys[-2000:], color='green', label='実データ')
ax0.legend()
ax0.set_title('RNN による予測結果')

ax1 = fig.add_subplot(gs[0, 1]) # middle
ax1.imshow(np.concatenate( (np.zeros((10,1)), y_hats)).reshape(64,64), cmap='gray')
ax1.set_title('再構成')
ax1.set_xticks([])
ax1.set_yticks([])

ax2 = fig.add_subplot(gs[0, 2]) # right
ax2.imshow(data.reshape(64,64), cmap='gray')
ax2.set_title('実データ')
ax2.set_xticks([])
ax2.set_yticks([])

plt.show()



# カルマンフィルタ Kalman filter 実習

In [ ]:
try:
    import filterpy
except ImportError:
    !pip install filterpy
    import filterpy

from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise


In [ ]:
N=0
zs = X[N] #.reshape(1,-1)
zs_mean = zs.mean(axis=0)
zs_cov = np.cov(zs)

kf = KalmanFilter(dim_x=1, dim_z=1) # zs.shape[0])
#kf = KalmanFilter(dim_x=zs.shape[0],dim_z=zs.shape[0])

kf.x = zs_mean
kf.F = zs_cov
kf.H = zs_cov

xs, Cov = [], []
for z in zs:
    kf.predict()
    kf.update(z)
    xs.append(kf.x)
    Cov.append(kf.P)

In [ ]:
plt.figure(figsize=(2,2))
plt.xticks([])
plt.yticks([])
plt.imshow(np.array(xs).reshape(64,64), cmap='gray')
plt.show()

# ARIMA 実習

### 自己相関 (AR) 和分 (I) 移動平均 MA モデル

ARIMA モデルとは自己相関 AR, 移動平均 MA, およびその和分 I をあわせた系列予測モデルである。

教科書には p, d, q としてそれぞれ AR, I, MA の次数を決める。
(p,d,q) = (1,0,0) であれば，1 次の自己相関となる。すなわち次式である：
$$
AR(1): y_{t} = \alpha y_{t-1} + c + \epsilon
$$
ここで時刻 $t$ の値 $y_{t}$ を予測する際に，直前の時刻 $t-1$ の値 $y_{t-1}$ の値から予測することになる。
$c$ は切片，$\alpha$ は回帰係数，$\epsilon$ は誤差である。

AR(2) すなわち 2 次の自己相関モデルであれば次式となる。
$$
AR(2): y_{t} = \alpha_{1}y_{t-1} + \alpha_{2}y_{t-2} + c + \epsilon
$$

ただし以下のような制約が存在する。
* AR(1)モデル: $−1<\phi_1<1$<br/>
* AR(2)モデル: $−1<\phi_2<1,\phi_1+\phi_2<1,\phi_2−\phi_1<1$

AR(m) m > 2 の場合制約は複雑になる。

$m$ 次の自己相関モデル AR(m) であれば，次式となる：
$$
y_{t} = \sum_{i=1}^{m}\alpha_{i}y_{i} + c + \epsilon
$$

ARIMA モデル，$d$ 階差分系列, $y_t-y_{t-d}$ を ARMA モデルで表現するのが ARIMA モデルである。
$$
ARIMA(p,d,q):y_{t} - y_{t-d} = c + \sum_{i=1}^{p}\phi_{i}y_{t-i} + \sum_{i=1}^{q}\theta_{i}\epsilon_{t-1}
$$

## 移動平均モデル

$$
y_t = c + \epsilon_t + \theta_1\epsilon_1+\cdots+\theta_{t-q}\epsilon_{t-q}
= c + \epsilon_t + \sum_{i=1}^{q}\theta_{i}\epsilon_{t-i},
$$
ここで $\epsilon_{t}$ は白色雑音である。上式を q 次の移動平均モデル MA(q) と呼ぶ。

MA(1) であれば次式となる：
$$
MA(1):y_t = c + \epsilon + \theta_1\epsilon_{t-1}
$$

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

x = X[0]
# ARIMA Model
for order in [(1,0,0), (1,1,0), (1,1,2)]:
    model = ARIMA(x, order=order)
    model_fit = model.fit()
    print(model_fit.summary())


In [ ]:
import pandas as pd

In [ ]:
# Plot residual errors
residuals = pd.DataFrame(model_fit.resid)
fig, ax = plt.subplots(1,2, figsize=(12,3))
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_predict
#plot_predict(model_fit)
#plt.show()

fig, axes = plt.subplots(1,2, figsize=(4,2))
axes[0].imshow(model_fit.predict().reshape(64,64), cmap='gray')
axes[1].imshow(x.reshape(64,64),cmap='gray')
plt.show()

In [ ]:
#model_fit.conf_int?

model_fit.standardized_forecasts_error.shape
#(steps=10)

# FFT 実習

In [ ]:
x = X[N]
y_fft = np.fft.fft(x)

y_fft1 = y_fft.copy()
y_fft2 = y_fft.copy()

threshold = 32
threshold = 64
# threshold = 128
# threshold = 512

# ローパスフィルタ
y_fft1.real[threshold:] = 0.
y_fft1.imag[threshold:] = 0.

# ハイパスフィルタ
y_fft2.real[:threshold] = 0.
y_fft2.imag[:threshold] = 0.
# 直交成分を足し合わせる
y_fft2.real[0] = y_fft.real[0]
y_fft2.imag[0] = y_fft.imag[0]

# 逆変換
y_ifft1 = np.fft.ifft(y_fft1)
y_ifft2 = np.fft.ifft(y_fft2)

plt.figure(figsize=(12,4))

#plt.plot(x, label='オリジナル')
plt.plot(y_ifft2.real, label='ハイパスフィルタ')
plt.plot(y_ifft1.real, label='ローパスフィルタ', lw=4)
plt.legend()
plt.show()

In [ ]:
figs, axes = plt.subplots(1,2,figsize=(4,2))
axes[0].imshow(y_ifft2.real.reshape(64,64),cmap='gray')
axes[0].set_title('ハイパスフィルタ')
axes[1].imshow(y_ifft1.real.reshape(64,64),cmap='gray')
axes[1].set_title('ローパスフィルタ')
plt.tight_layout()
plt.show()

In [ ]:
# 2d fft
y_fft2d = np.fft.fft2(x.reshape(64,64))

# 逆変換
y_ifft2d_1 = np.fft.ifft2(y_fft2d)
#y_ifft2 = np.fft.fft2.ifft(y_fft2d)

#np.set_printoptions(precision=3)
plt.figure(figsize=(3,3))
plt.imshow(y_ifft2d_1.real, cmap='gray') #.shape

In [ ]:
y_fft2d_1 = y_fft2d.copy()
y_fft2d_2 = y_fft2d.copy()

threshold = 32
threshold = 64
# threshold = 128
#threshold = 1024

# ローパスフィルタ
y_fft2d_1.real[threshold:, threshold:] = 0.
y_fft2d_1.imag[threshold:, threshold:] = 0.

# ハイパスフィルタ
y_fft2d_2.real[:threshold, :threshold] = 0.
y_fft2d_2.imag[:threshold, :threshold] = 0.
# 直交成分を足し合わせる
y_fft2d_2.real[0,0] = y_fft2d.real[0,0]
y_fft2d_2.imag[0,0] = y_fft2d.imag[0,0]

# 逆変換
y_ifft2d_1 = np.fft.ifft2(y_fft2d_1)
y_ifft2d_2 = np.fft.ifft2(y_fft2d_2)

figs, axes = plt.subplots(1,2,figsize=(4,2))
axes[0].imshow(y_ifft2d_1.real.reshape(64,64),cmap='gray')
axes[0].set_title('ハイパスフィルタ')
axes[1].imshow(y_ifft2d_2.real.reshape(64,64),cmap='gray')
axes[1].set_title('ローパスフィルタ')
plt.tight_layout()
plt.show()

In [ ]:
N = 4096 >> 0
np.random.seed(42)
epsilons = [np.random.randn() for _ in range(N+3)]

theta0, theta1, theta2 = 1.0, 0.2, 0.5
X = [(epsilons[i],epsilons[i+1], epsilons[i+2]) for i in range(N)]
y = [theta0 * _x[2] + theta1 * _x[1] + theta2 * _x[0] for _x in X]
#len(y), y[:5]
plt.figure(figsize=(12,3))
plt.title('データ')
plt.plot(y)

In [ ]:
len(y)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import statsmodels
print(f'statsmodels.__version__:{statsmodels.__version__}')
model = ARIMA(y, order=(0, 0, 2))
result = model.fit()
print(result.summary())

In [ ]:
predct = result.predict()

# 残差 plot
plt.figure(figsize=(20,5))
plt.plot(result.resid)
plt.axhline(0, color='red', linestyle='--')
plt.show()



In [ ]:
from statsmodels.tsa.stattools import pacf, acf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
#len(y), len(pacf(y,lags=1))
plot_acf(y, lags=100)
plt.show()

In [ ]:
y = X[0]

model = ARIMA(y, order=(1, 0, 1))
result = model.fit()
print(result.summary())

In [ ]:
X.shape
x = X[0]
# x.shape, zs.shape
# print(zs[:10])
# plt.plot(zs)
# plt.plot(estimates)
# plt.show()

fig, axes = plt.subplots(1,2,figsize=(4,2))
axes[0].set_title('KF estimates')
axes[0].imshow(np.array(estimates).reshape(64,64), cmap='gray')
axes[0].axis(False)
axes[1].imshow(x.reshape(64,64), cmap='gray')
axes[1].axis(False)
axes[1].set_title('Original')
plt.tight_layout()
plt.show()

In [ ]:
#from filterpy.stats import gaussian
X.shape
N = 70
zs = X[N]

from collections import namedtuple
gaussian = namedtuple('Gaussian', ['mean', 'var'])
gaussian.__repr__ = lambda s: '𝒩(μ={:.3f}, 𝜎²={:.3f})'.format(s[0], s[1])

def update(likelihood, prior):
    posterior = likelihood * prior
    return normalize(posterior)

def update(prior, measurement):
    x, P = prior        # 事前分布の平均と分散
    z, R = measurement  # 観測値の平均と分散

    y = z - x        # 残差
    K = P / (P + R)  # カルマンゲイン

    x = x + K*y      # 事後分布の平均
    P = (1 - K) * P  # 事後分布の分散
    return gaussian(x, P)

def predict(posterior, movement):
    x, P = posterior # 事後分布の平均と分散
    dx, Q = movement # 移動量の平均と分散
    x = x + dx
    P = P + Q
    return gaussian(x, P)

voltage_std = .13
x = gaussian(.5, 1.) # 初期状態
process_var = 0.05**2
#process_var = 1.**2
process_model = gaussian(0., process_var)

N = 50
# zs = [volt(actual_voltage, voltage_std) for i in range(N)]
ps = []
estimates = []

for z in zs:
    prior = predict(x, process_model)
    x = update(prior, gaussian(z, voltage_std**2))

    # グラフにするために記録する
    estimates.append(x.mean)
    ps.append(x.var)


plt.figure(figsize=(14,3))
plt.plot(zs, label='zs')
plt.plot(estimates, label='estimates')
plt.legend()
#plt.ylim(16, 17)
plt.show()

plt.plot(ps)
plt.title('Variance')
print('Variance converges to {:.3f}'.format(ps[-1]))

fig, axes = plt.subplots(1,2,figsize=(3,2))
axes[0].set_title('KF estimates')
axes[0].imshow(np.array(estimates).reshape(64,64), cmap='gray')
axes[0].axis(False)
axes[1].imshow(zs.reshape(64,64), cmap='gray')
axes[1].axis(False)
axes[1].set_title('Original')
plt.tight_layout()
plt.show()

In [ ]:
zs

# 機械学習手法による顔認識

## データの分割，訓練データとテストデータ

データを 2 分割して，訓練データセットとテストデータセットに分割します。
分割した訓練データセットでモデルのパラメータを学習し，しかる後に，テストデータセットで，その汎化性能を評価します。
このとき，テストデータセットでの性能が高いモデルが良いモデルということになります。

オリベッティ顔データセットには， 各被験者の 10 枚の顔画像が含まれています。
このうち，例えば 90% を訓練データとし，10% をテストデータとして使用することを考えます。
各顔データの訓練画像とテスト画像の数が同じになるように stratify 機能を使用してます。
したがって，各被験者には 9 枚の訓練用画像と 1 枚のテスト用画像が用意されることになります。
訓練データとテストデータの割合は split_ratio 変更することができます。


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

#from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

#split_ratio = 0.3 としているので，訓練データ対テストデータが 7:3 となる
split_ratio = 0.3
X_train, X_test, y_train, y_test=train_test_split(X, y_sex, test_size=split_ratio, stratify=y, random_state=42)
#X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=split_ratio, stratify=y, random_state=42)
print(f'X_train 訓練画像のサイズ: {X_train.shape}')
print(f'y_train 教師信号データのサイズ: {y_train.shape}')

print(f'X_test 検証画像のサイズ: {X_test.shape}')
print(f'y_test 教師信号データのサイズ: {y_test.shape}')

## ロジスティック回帰

In [ ]:
params = {'max_iter':10 ** 3,
          'C':1e3,
          #'penalty':'l2'
         }

model = LogisticRegression(**params)
model.fit(X_train, y_train)    # 訓練データを用いて線形判別分析モデルを訓練\n",
y_hat = model.predict(X_test)  # テストデータを使って予測を行い結果を y_hat に格納\n",
print(f"ロジスティック回帰を用いた分類精度: {metrics.accuracy_score(y_test, y_hat):.3f}")
print(f'分類報告:\n{metrics.classification_report(y_test,y_hat)}')

# 混同行列の表示
print('混同行列:\n', metrics.confusion_matrix(y_test,y_hat))
# plt.figure(figsize=(8,6))
# sns.heatmap(metrics.confusion_matrix(y_test, y_hat))

## 線形判別分析


In [ ]:
params = {#'max_iter':10 ** 3,
          #'C':1e3,
          #'penalty':'l2'
         }

model = LinearDiscriminantAnalysis(**params)
model.fit(X_train, y_train)    # 訓練データを用いて線形判別分析モデルを訓練
y_hat = model.predict(X_test)  # テストデータを使って予測を行い結果を y_hat に格納
print(f"線形判別分析を用いた分類精度: {metrics.accuracy_score(y_test, y_hat):.3f}")
print(f'分類報告:\n{metrics.classification_report(y_test,y_hat)}')

# 混同行列の表示
print('混同行列:\n', metrics.confusion_matrix(y_test,y_hat))
# plt.figure(figsize=(8,6))
# sns.heatmap(metrics.confusion_matrix(y_test, y_hat))

## サポートベクターマシン


In [ ]:
params = {'max_iter':10 ** 3,
          'C':1e3,
          #'penalty':'l2'
         }

model = SVC(**params)
model.fit(X_train, y_train)    # 訓練データを用いて線形判別分析モデルを訓練
y_hat = model.predict(X_test)  # テストデータを使って予測を行い結果を y_hat に格納
print(f"サポートベクターマシン用いた分類精度: {metrics.accuracy_score(y_test, y_hat):.3f}")
print(f'分類報告:\n{metrics.classification_report(y_test,y_hat)}')
# print(f'適合率 precision score:{metrics.precision_score(y_test, y_hat):.3f}')
# print(f'再現率 recall  score  :{metrics.recall_score(y_test, y_hat):.3f}')
# print(f'F1 値  F1 score       :{metrics.f1_score(y_test, y_hat):.3f}')

# 混同行列の表示
print('混同行列:\n', metrics.confusion_matrix(y_test,y_hat))
# plt.figure(figsize=(8,6))
# sns.heatmap(metrics.confusion_matrix(y_test, y_hat))

## 3 手法比較


In [ ]:
# 40 名の顔認識データをもちいて 3 手法を比較
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=split_ratio, stratify=y, random_state=42)

# 共通パラメータ
params = {'max_iter':10 ** 4,
          #'C':1e3,
          #'penalty':'l2'
         }

# 3 つのモデルの定義
svc_model = SVC(**params)
logistic_model = LogisticRegression(**params)
lda_model = LinearDiscriminantAnalysis()

# 出力する図のサイズを定義
fig, _axes = plt.subplots(ncols=3, nrows=1, figsize=(12,4), constrained_layout=True)

# 3 つのモデルをそれぞれ実行して結果を描画
for i, (model_name, model) in enumerate([('サポートベクタマシン',svc_model),
                                         ('ロジスティック回帰',logistic_model),
                                         ('線形判別分析',lda_model)]):
    model.fit(X_train, y_train)
    y_hat = model.predict(X_test)
    y_hat = model.predict(X_test)  # テストデータを使って予測を行い結果を y_hat に格納

    _axes[i].imshow(metrics.confusion_matrix(y_test,y_hat), cmap='gray')
    _axes[i].set_title(f'{model_name}:分類精度:{metrics.accuracy_score(y_test,y_hat):.2f}')
    _axes[i].axis('off')
    print(f'分類報告:\n{metrics.classification_report(y_test,y_hat)}')
    # print(f'適合率 precision score:{metrics.precision_score(y_test, y_hat, average='macro'):.3f}')
    # print(f'再現率 recall  score  :{metrics.recall_score(y_test, y_hat, average='macro'):.3f}')
    # print(f'F1 値  F1 score       :{metrics.f1_score(y_test, y_hat, average='macro'):.3f}')

plt.show()

## リーブ・ワン・アウト 交差検証

オリベッティ顔データセットには，各被験者に対して 10 枚の顔画像が含まれています。
これは， 機械学習モデルの学習やテストには少ない数です。

クラスの例が少ない機械学習モデルをよりよく評価するために，採用される交差検証法にリーブ・ワン・アウト leave-one-out (LOO) 交差検証法があります。
LOO 法では，あるクラスのサンプルのうち 1 つだけをテストに使用します。
他のサンプルは訓練に使用します。 この手順を， 全サンプルを一度づつテストに使用して繰り返さします。

In [ ]:
%%time
from sklearn.model_selection import LeaveOneOut

loo_cv = LeaveOneOut()
model = LogisticRegression(**params)
cv_scores = cross_val_score(model, X_train, y_train, cv=loo_cv)

print(f"{model.__class__.__name__} リーブ・ワン・アウト交差検証法による平均得点:{cv_scores.mean():.3f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# シグモイド関数の描画
# logits = [_x/(1.-_x) for _x in x]
# plt.plot(x, logits)
# plt.show()

# plt.plot(x, np.log(logits))
# plt.show()

# x = np.arange(-10, 10, 0.01)
# sigmoids = [1./(1.+np.exp(-_x)) for _x in x]
# plt.plot(x, sigmoids)
# plt.show()

# x = np.arange(-10, 10, 0.01)
# sigmoids = [1./(1.+np.exp(-_x)) for _x in x]
# plt.plot(x, sigmoids, label="sigmoid", c='red')
# plt.plot(x, np.tanh(x), label="tanh", c='green')
# plt.legend()
# plt.show()

## 交差検証

In [ ]:
%%time
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

name = 'ロジスティック回帰'
model = LogisticRegression(**params)
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X_train, y_train, cv=kfold)
print(f"{name} 平均交差検証得点: {cv_scores.mean():.2f}")

## ハイパーパラメータの調整: GridSearcCV

モデルの汎化性能向上のために GridSearchCV を行います。
ロジスティック回帰分類器のハイパーパラメータを調整してみます。

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneOut

params2={'penalty':['l1', 'l2'],
        'C':np.logspace(0, 4, 10),
        'max_iter': [10 ** 4],
       }
model = LogisticRegression()
loo_cv = LeaveOneOut()
gridSearchCV = GridSearchCV(model, params2, cv=loo_cv)
gridSearchCV.fit(X_train, y_train)
print("Grid search fitted..")
print(gridSearchCV.best_params_)
print(gridSearchCV.best_score_)
print(f"グリッドサーチによる交差妥当性得点:{gridSearchCV.score(X_test, y_test):.3f}")

# PCA 固有顔 Eigenfaces

In [ ]:
# X を n 行 m 列の行列として，列方向 m x m の相関係数行列を求める
# すなわち各画素ごとの相関係数行列を計算する
# オリベッティ顔データの場合 64 x 64 = 4096 画素分のデータなので,相関係数行列は 4096 x 4096 の大きさとなる
#np.set_printoptions(precision=3)
np.set_printoptions(formatter={'float': '{:.2f}'.format})

x_mean = np.mean(X, axis=0)   # 各列の平均値を計算
_X = X - x_mean               # 各列の平均値を減じて平均偏差ベクトルとする
Cov = _X.T @ _X / _X.shape[0] # 共分散行列
_X_std = np.std(_X, axis=0)     # 各列の標準偏差
R = Cov / np.outer(_X_std.T, _X_std) # 共分散行列の各列を対応する標準偏差の積で除して相関係数行列にする
print(f'R.shape:{R.shape}')        # 確認用 相関係数行列のサイズ
print(f'相関係数行列 R:\n{R[8:15,8:15]}')           # 確認用 相関係数行列の最初の 3 行 3 列を表示する

R2 = np.corrcoef(X.T)              # 上記を一行で行う numpy コマンド
print(f'相関係数行列 R2:\n{R2[8:15,8:15]}')         # 結果の表示

In [ ]:
# 上セルとは異なりデータ行列 X (n 行 m 列) の行方向 n x n の相関係数行列を求める
# この場合 400 画像 x 400 画像 (400 は 40 人分の画像で各人 10 枚の画像)
import seaborn as sns  # ヒートマップ描画のために使用

plt.figure(figsize=(10,10))
sns.heatmap(np.corrcoef(X), center=0, vmin=-1., vmax=1., square=True, cmap='gnuplot2')
plt.show()

In [ ]:
# 平均画像を描画
x_mean = np.mean(X, axis=0)   # 各列の平均値を計算
print(f'x_mean.min():{x_mean.min():.2f}',
      f'x_mean.max():{x_mean.max():.2f}')
#print(X.min(), X.max())
plt.figure(figsize=(2.5,2.5))
plt.title(f'平均顔, 最小値:{x_mean.min():.2f}, 最大値:{x_mean.max():.2f}')
plt.imshow(x_mean.reshape(64,64), cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
# いくつかの画像に対して，平均顔からの差分を画像化して表示してみる
idxes = [0, 10, 20, 30, 40, 50, 60, 70, 80]
fig, axes = plt.subplots(nrows=1, ncols=len(idxes), figsize=(len(idxes) * 1.2, 1.5))
for i in range(0,len(idxes),1):
    idx = idxes[i]
    #print(f'idx:{idx}')
    axes[i].imshow((X[idx] - x_mean).reshape(64,64), cmap='gray')
    axes[i].set_title(f'idx:{idx}')
    axes[i].axis('off')

plt.show()

In [ ]:
%%time
class PCA():
    def __init__(self, X:np.array, n_dim:int=500):
        self.X = X

        N, M = X.shape
        if N < M:
            self.n_dim = N
        else:
            self.n_dim = M

        if self.n_dim > n_dim:
            self.n_dim = n_dim
        self.mean = np.mean(X, axis=0)          # 行列 X の各列ごとの平均を求める
        self._X = X - self.mean                 # 各列ごとの平均を引いた 平均偏差行列 _X
        self.Corr = np.dot(self._X.T, self._X)  # 各列ごとの分散共分散行列 Corr
        self.Eigenvalues, self.Eigenvectors = np.linalg.eig(self.Corr)  # 固有値問題を解く

        self.Eigenvalues = self.Eigenvalues[0:self.n_dim].copy()
        self.Eigenvectors = self.Eigenvectors[:,0:self.n_dim].copy()
        self.projections = np.dot(self._X, self.Eigenvectors)

    def _reconstruct(self, start:int=0, end:int=10):
        return np.dot(self.projections[:,start:end], self.Eigenvectors[:,start:end].T) + self.mean


_PCA = PCA(X=X)
Eigenvalues, Eigenvectors, mean = _PCA.Eigenvalues, _PCA.Eigenvectors, _PCA.mean
print(f'Eigenvalues.shape:{Eigenvalues.shape}',
      f'\nEigenvectors.shape:{Eigenvectors.shape}',
      f'\nmean.shape:{mean.shape}')

plt.figure(figsize=(2,2))
plt.title('平均顔')
plt.imshow(mean.reshape(64,64), cmap='gray')
plt.show()

In [ ]:
# いくつかの画像に対して，平均顔からの差分を画像化して表示してみる
idxes = [0, 10, 20, 30, 40, 50, 60, 70, 80]
n_idx = 12
idxes = sorted(np.random.permutation(np.arange(X.shape[0]))[:n_idx])
fig, axes = plt.subplots(nrows=1, ncols=len(idxes), figsize=(len(idxes) * 1.2, 1.5))
for i in range(0,len(idxes),1):
    idx = idxes[i]
    #print(f'idx:{idx}')
    axes[i].imshow((X[idx] - x_mean).reshape(64,64), cmap='gray')
    axes[i].set_title(f'idx:{idx}', size=8)
    axes[i].axis('off')

plt.suptitle('いくつかの画像に対して，平均顔からの差分を画像化', size=10)
plt.show()

## 固有値のプロット (降順)

In [ ]:
# 大きい方から上位 N 個の固有値をプロットしてみる
N = 200
plt.plot(Eigenvalues[:N])
plt.title(f'固有値のプロット: max:{_PCA.n_dim}')


N_pca = 10
fix, axes = plt.subplots(nrows=1, ncols=N_pca, figsize=(N_pca * 1.4, 1.4))
for i in range(N_pca):
    axes[i].imshow(_PCA.Eigenvectors[:,i].reshape(64,64), cmap="gray")
    axes[i].set_title(f'第 {i+1} 固有値に対応\nする固有ベクトル', size=7)
    axes[i].axis('off')

plt.show()

## 固有ベクトルの相関係数行列を可視化


In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(np.corrcoef(_PCA.Eigenvectors.T)[:400,:400], center=0, vmin=-1., vmax=1., square=True, cmap='gnuplot2')
plt.title('固有ベクトルの相関係数行列を可視化')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(_PCA.Eigenvectors.T @ _PCA.Eigenvectors, center=0, vmin=-1., vmax=1., square=True, cmap='gnuplot2')
plt.title('固有ベクトルの積を可視化')
plt.show()

In [ ]:
print('固有値は各行ごとに正規化されている。かつ，各固有値は直交している。すなわち対角要素が 1 で非対角要素は 0 である')
print((_PCA.Eigenvectors.T @ _PCA.Eigenvectors)[:5,:5])

print('固有値は各列ごとではない。その証拠に列ごとに計算してみると以下のようになる')
print((_PCA.Eigenvectors @ _PCA.Eigenvectors.T)[:5,:5])

## PCA による次元圧縮の結果を表示


In [ ]:
# PCA による次元圧縮の結果を表示

fig, axes = plt.subplots(ncols=3, nrows=1, figsize=(14,4))
axes[0].scatter(_PCA.projections[:,0], _PCA.projections[:,1], s=1)
axes[0].set_xlabel('第 1 主成分')
axes[0].set_ylabel('第 2 主成分')
for i in range(_PCA.projections.shape[0]):
    axes[0].annotate(str(y[i]), (_PCA.projections[i,0], _PCA.projections[i,1]))

axes[1].scatter(_PCA.projections[:,1], _PCA.projections[:,2], s=1)
axes[1].set_xlabel('第 2 主成分')
axes[1].set_ylabel('第 3 主成分')
for i in range(_PCA.projections.shape[0]):
    axes[1].annotate(str(y[i]), (_PCA.projections[i,1], _PCA.projections[i,2]))

axes[2].scatter(_PCA.projections[:,0], _PCA.projections[:,3], s=1)
axes[2].set_xlabel('第 1 主成分')
axes[2].set_ylabel('第 4 主成分')
for i in range(_PCA.projections.shape[0]):
    axes[2].annotate(str(y[i]), (_PCA.projections[i,0], _PCA.projections[i,3]))

plt.show()

## PCA による固有顔の再構成

In [ ]:
# 表示する画像番号リストを idexes で宣言
idxes = [0, 10, 20, 40, 60, 70]
idxes = [70, 71, 72, 73, 74, 75]

# 乱数を発生させて n_idx 個のデータ画像をランダムに選んで idxes に格納する
n_idx = 10
idxes = sorted(np.random.permutation(np.arange(X.shape[0]))[:n_idx])

# PCA による画像の再構成時に何個の固有ベクトルを重ねるを指定するリスト ends
ends = [1, 10, 20, 40, 80, 160, 320]
ends = [1, 10, 20, 30, 40, 50, 100, 150, 200, 250, 300]

# 表示する画像サイズの宣言 figsize の幅と高さを指定,単位は歴史的経緯からインチ
fig, axes = plt.subplots(nrows=len(idxes), ncols=len(ends), figsize=(1.4 * len(ends), 1.6 * len(idxes)))
for i, idx in enumerate(idxes):
    for j, end in enumerate(ends):
        axes[i][j].axis('off')
        if j == 0:
            axes[i][j].set_title(f'画像番号:{idx}')
        else:
            axes[i][j].set_title(f'{end} 次元まで')
        axes[i][j].imshow((_PCA.projections[idx,0:end] @ _PCA.Eigenvectors[:,0:end].T + _PCA.mean).reshape(64,64), cmap='gray')

plt.show()


## 固有顔 (PCA) を用いた分類

In [ ]:
# 固有顔 (PCA) を用いて 40 名の顔認識データをもちいて 3 手法を比較
X_train, X_test, y_train, y_test=train_test_split(_PCA.projections, y, test_size=split_ratio, stratify=y, random_state=42)
#X_train, X_test, y_train, y_test=train_test_split(_PCA.projections, y_sex, test_size=split_ratio, stratify=y, random_state=42)

# 共通パラメータ
params = {'max_iter':10 ** 4,
          #'C':1e3,
          #'penalty':'l2'
         }

# 3 つのモデルの定義
svc_model = SVC(**params)
logistic_model = LogisticRegression(**params)
lda_model = LinearDiscriminantAnalysis()

# 出力する図のサイズを定義
fig, _axes = plt.subplots(ncols=3, nrows=1, figsize=(12,4), constrained_layout=True)

# 3 つのモデルをそれぞれ実行して結果を描画
for i, (model_name, model) in enumerate([('サポートベクタマシン',svc_model),
                                         ('ロジスティック回帰',logistic_model),
                                         ('線形判別分析',lda_model)]):
    model.fit(X_train, y_train)

    y_hat = model.predict(X_test)  # テストデータを使って予測を行い結果を y_hat に格納

    _axes[i].imshow(metrics.confusion_matrix(y_test,y_hat), cmap='gray')
    _axes[i].set_title(f'{model_name}:分類精度:{metrics.accuracy_score(y_test,y_hat):.2f}')
    _axes[i].axis('off')

    print(model_name, '\n', metrics.confusion_matrix(y_test,y_hat))
    print(model_name, '\n', metrics.classification_report(y_test,y_hat))

plt.show()

## tSNE によるプロット

In [ ]:
from sklearn.manifold import TSNE
X_tsne = TSNE(n_components=2, learning_rate='auto',
              init='random', perplexity=3).fit_transform(X)

PCA_tsne = TSNE(n_components=2, learning_rate='auto',
              init='random', perplexity=3).fit_transform(_PCA.projections)


In [ ]:
#plt.figure(figsize=(12,6))
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,6))

axes[0].scatter(X_tsne[:,0], X_tsne[:,1], s=1)
for i in range(X_tsne.shape[0]):
    axes[0].annotate(str(y[i]), (X_tsne[i,0], X_tsne[i,1]))
axes[0].set_title('tSNE プロット')

axes[1].scatter(PCA_tsne[:,0], PCA_tsne[:,1], s=1)
for i in range(PCA_tsne.shape[0]):
    axes[1].annotate(str(y[i]), (X_tsne[i,0], X_tsne[i,1]))
axes[1].set_title('tSNE プロット')
plt.show()

# フィッシャー顔 Fisherfaces

線形判別分析は、R.A.フィッシャー 卿によって発明された。
フィッシャーは 1936 年の論文 "The use of multiple measurements in classificationonomic problems" の中で、アヤメの分類に用いることに成功した。
<!-- しかし、主成分分析 (PCA) がこれほどうまくいったのに、なぜ別の次元削減法が必要なのだろうか？-->
PCA はデータの全分散を最大化する特徴の線形結合を見つける。
これは強力な方法だが、クラスを考慮しないため、成分を捨てるときに多くの識別情報が失われる可能性がある。
分散が外部ソースによって生成される状況を想像してみよう。
PCA によって同定された成分は、必ずしも識別情報を全く含まないので、投影されたサンプルは互いに混ざり合い、分類は不可能になる。
クラス間を最もよく分離する特徴の組み合わせを見つけるために、線形判別分析はクラス間のばらつきとクラス内のばらつきの比率を最大化する。
考え方は単純で、同じクラスは互いに密に集まり、異なるクラスはできるだけ離れるべきである。
このことは Belhumeur_Hespanha_Kriegman も認識しており、彼らは [3] で顔認識に判別分析を適用した。
<!-- The Linear Discriminant Analysis was invented by the great statistician Sir R. A. Fisher, who successfully used it for classifying owers in his 1936 paper The use of multiple measurements in taxonomic problems [8].
But why do we need another dimensionality reduction method, if the Principal Component Analysis (PCA) did such a good job?
The PCA  finds a linear combination of features that maximizes the total variance in data.
While this is clearly a powerful way to represuccsent data, it doesn't consider any classes and so a lot of discriminative information may be lost when throwing components away.
Imagine a situation where the variance is generated by an external source, let it be the light.
The components identified by a PCA do not necessarily contain any discriminative information at all, so the projected samples are smeared together and a classification becomes impossible.
In order to  nd the combination of features that separates best between classes the Linear Discriminant Analysis maximizes the ratio of between-classes to within-classes scatter.
The idea is simple: same classes should cluster tightly together, while different classes are as far away as possible from each other.
This was also recognized by Belhumeur, Hespanha and Kriegman and so they applied a Discriminant Analysis to face recognition in [3]. -->

* [3] Belhumeur, P. N., Hespanha, J., and Kriegman, D. Eigenfaces vs. Fisherfaces: Recognition using class specific linear projection. IEEE Transactions on Pattern Analysis and Machine Intelligence 19, 7 (1997), 711-720.
* [4] Brunelli, R., and Poggio, T. Face recognition through geometrical features. In European Conference on Computer Vision (ECCV) (1992), pp. 792-800.
* [5] Cardinaux, F., Sanderson, C., and Bengio, S. User authentication via adapted statistical models of face images. IEEE Transactions on Signal Processing 54 (January 2006), 361-373.
* [6] Chiara Turati, Viola Macchi Cassia, F. S., and Leo, I. Newborns face recognition: Role of inner and outer facial features. Child Development 77, 2 (2006), 297-311.
* [7] Duda, R. O., Hart, P. E., and Stork, D. G. Pattern Classification (2nd Edition), 2 ed. November 2001.
* [8] Fisher, R. A. The use of multiple measurements in taxonomic problems. Annals Eugen. 7 (1936), 179-188.

### アルゴリズムの説明
<!--2.3.1 Algorithmic Description-->

$X$ を $c$ 個のクラスから抽出したランダムベクトルとする：
<!--Let $X$ be a random vector with samples drawn from c classes: -->
$$\tag{8}
X = \left\{X_1,X_2,\ldots,X_{c}\right\}
$$
$$\tag{9}
X_i = \left\{x_1, x_2,\ldots, x_{n}\right\}
$$

分散行列 $S_B$ (級間分散) と $S_W$ (級内分散) は次式のように定義される：
<!-- The scatter matrices $S_B$ and $S_W$ are defined as: -->
$$\tag{10}
S_B = \sum_{i=1}^{c} N_i (\mu_i - \mu)(\mu_i - \mu)^{\top},
$$
$$\tag{11}
S_W = \sum_{i=1}^{c} \sum_{x \in X_i} (x - \mu_i)(x - \mu_i)^{\top},
$$
ここで $\mu$ は全平均:<!--where $\mu$ is the total mean:-->
$$\tag{12}
\mu = \frac{1}{N} \sum_{i=1}^{N} x_i,
$$
さらに $\mu_i$ は群内平均:<!--And $\mu_i$ is the mean of class $i\in\left\{1,\ldots,C\right\}$:-->
$$\tag{13}
\mu_i = \frac{1}{\left|X_i\right|} \sum_{x_{j} \in X_{i}} x_{j}.
$$
フィッシャーの古典的アルゴリズムは、クラス分離可能性基準を最大化する射影行列 $W$ を探す：
<!-- Fisher's classic algorithm now looks for a projection matrix $W$, that maximizes the class separability criterion: -->
$$\tag{14}
W_{opt}=\arg\max_{W} \frac{\left|W^{\top} S_B W\right|}{\left|W^{\top} S_W W\right|},
$$
[3] に従って、この最適化問題の解は、一般固有値問題を解くことによって与えられる：
<!-- Following[3], a solution for this optimization problem is given by solving the Genral Eigenvalue Problem: -->
$$\tag{15}\begin{aligned}
S_{B}\nu_i &= \lambda_i S_{W}\nu_i,\\
S_{W}^{-1} S_{B} \nu_i &= \lambda_i \nu_i,
\end{aligned}$$

解決すべき問題が 1 つ残っている：
$S_W$ のランクは最大でも $(N-c)$ であり，$N$ 個のサンプルと $c$ 個のクラスが存在する。
パターン認識問題では，サンプル数 $N$ は入力データの次元 (画素数) より小さいことがほとんどなので，分散行列 $S_W$ は特異行列になる ([2]を参照)。
[3] では、データに対して主成分分析を実行し、サンプルを (N-c) 次元空間に射影することでこれを解決した。
$S_W$ が特異でなくなったので、線形判別分析が縮小データに対して実行される。
最適化問題は次のように書き換えられる:
<!-- There's one problem left to solve:
The rank of S_W is at most (N-c), with N samples and c classes.
In pattern recognition problems the number of samples N is almost always smaller than the dimension of the input data (the number of pixels), so the scatter matrix SW becomes singular (see [2]).
In [3] this was solved by performing a Principal Component Analysis on the data and projecting the samples int the (N-c)-dimensional space.
A Linear Discriminant Analysis is then performed on the reduced data, because S_W is not singular anymore.
The optimization problem is can be rewritten as: -->
$$\tag{16}
W_{pca}=\arg\max_{W} \left|W^{\top} S_T W\right|
$$

$$\tag{17}
W_{fld} = \arg\max_W \frac{\left|W^{\top} W_{pca}^{\top} S_B W_{pca}W\right|}{\left|W^{\top} W_{pca}^{\top}S_W W_{pca}W\right|},
$$
サンプルを (c-1) 次元空間に投影する変換行列 $W$ は次のように与えられる：
<!-- The transformation matrix $W$, that projects a sample into the (c-1)-dimensional space is then given by:-->
$$\tag{18}
W = W_{fld}^{\top}W_{pca}^{\top}.
$$

最後に注意点：
$S_W$ と $S_B$ とは対称行列であるが，対称行列同士の積は必ずしも対称ではないので，一般行列用の固有値ソルバを使う必要がある。
OpenCV の `cv:eigen` は，現在のバージョンでは対称行列に対してのみ動作する。
非対称行列に対しては，特異値の固有値は等価ではないので，特異値分解（SVD）ソルバーを利用することはできない．
<!-- one final note:
Although $S_W$ and $S_B$ are symmetric matrices, the product of two symmetric matrices is not necessarily symmetric; thus, so you have to use an eigenvalue solver for general matrices.
OpenCV's `cv:eigen` only works for symmetric matrices in it scurrent version; since eigenvlues of singular values are not equivalent for non-symmetric matrices you can not use a Singular Value Decomposition (SVD) eigher. -->



## 固有顔 Eigenfaces

与えられた画像表現の問題は、その高次元性である。
2次元の $p\times q$ の濃淡画像は $m=pq$ 次元のベクトル空間にまたがるので、$100\times100$ 画素の画像はすでに 10,000 次元の画像空間にある。
これはどんな計算をするにも多すぎるが、すべての次元が本当に我々にとって有用なのだろうか？
我々はデータに分散がある場合にのみ判断を下すことができるので、我々が探しているのは情報の大部分を占める成分である。
主成分分析 (PCA) は、カール・ピアソン(1901) とハロルド・ホテリング (1933) によって独自に提案されたもので，相関している可能性のある変数の集合を，より小さな相関していない変数の集合に変えるものである。
このアイデアは、高次元のデータ集合は、しばしば相関のある変数によって記述され、したがって、いくつかの意味のある次元だけが情報の大部分を占めるというものである。
PCA は、主成分と呼ばれるデータ中の最大の分散を持つ方向を見つける。
<!-- The problem with the image representation we are given is its high dimensionality.
Two-dimensional pxq grayscale images span a m=pq-dimensional vector space, so an image with 100x100 pixels lies in a 10,000-dimensional image space already.
That's way too much for any computations, but are all dimensions really useful for us?
We can only make a decision if there's any variance in data, so what we are looking for are the components that account for most of the information.
The Principal Component Analysis (PCA) was independently proposed by Karl Pearson (1901) and Harold Hotelling (1933) to turn a set of possibly correlated variables into a smaller set of uncorrelated variables.
The idea is that a high-dimensional dataset is often described by correlated variables and therefore only a few meaningful dimensions account for most of the information.
The PCA method finds the directions with the greatest variance in the data, called principal components. -->


### アルゴリズム<!--Algorithmic Description-->

$X=\left\{x_1,x_2,\ldots,x_n\right\}$ は $x_i\in\mathbb{R}^{d}$ からの確率ベクトルとする:
<!-- Let $X=\left\{x_1,x_2,\ldots,x_n\right\}$ be a random vector with observation $x_i\in\mathbb{R}^{d}$. -->

1. Compute the mean $\mu$
$$\tag{1}
\mu=\frac{1}{n}\sum_{i=1}^{n}x_i.
$$
2. Compute the Covariance matrix $C$
$$\tag{2}
C=\frac{1}{n}\sum_{i=1}^{n}(x_i-\mu)(x_i-\mu)^{\top}.
$$
3. Compute the eigenvalues $\lambda_i$ and eigenvectors $\nu_i$ of $C$
$$\tag{3}
C\nu_i=\lambda_i\nu_i, \text{\hspace{1cm} $i=1,2,\ldots,n$}
$$
4. Order the eigenvalues descending by their eigenvalue.
The $k$ principla components are the eigenvectors corresponding to the $k$ largest eigenvalues.

観測ベクトル $x$ の $k$ 個の主成分は次式で与えられる：<!-- The k principal components of the observed vector $x$ are then given by: -->

$$\tag{4}
y = W^{\top}(x-\mu),
$$

ここで $W=\left(v_1,v_2,\ldots,v_k\right)$.
<!--where $W=left(v_1,v_2,\ldots,v_k\right)$.-->
PCA 基底からの再構成は次式:<!--The reconstruction from the PCA basis is given by:-->
$$\tag{5}
x= Wy + \mu.
$$
そして固有顔法は、次のようにして顔認識を行う：<!-- The Eigenfaces method then performs face recognition by: -->
1. すべての訓練サンプルを PCA の部分空間に射影する  (式 (4) )。
2. クエリ画像を PCA 部分空間に射影する (リスト 5 を使用)。
3. 投影された訓練サンプルと投影されたクエリ画像の間の最近傍を見つける。

<!-- 1. Projecting all training samples into the PCA subspace (using Equation 4).
2. Projecting the query image into the PCA subspace (using Listing 5).
3. Finding the nearest neighbor between the projected training samples and the projected query image. -->

まだ、解決しなければならない問題が 1 つ残っている。
100x100 画素の画像が 400 枚与えられたとする。
主成分分析（PCA）は共分散行列 $C=XX^{\top}$ を解くが、この例では size(X)=$10000\times400$ である。
結局 10000x10000 の行列になり、ざっと 0.8GB になる。
この問題を解くのは現実的ではないので、トリックを適用する必要がある。
線形代数の授業で、$M>N$ の $M\times N$ 行列は $N-1$ 個の非ゼロ固有値しか持てないことを知っているだろう。
そこで、代わりにサイズ $N\times N$ の固有値分解 $S=X^{\top}X$ を取ることができる：
<!-- Still there is one problem left to solve.
Imagin we are given 400 images sized 100x100 pixels.
The Principal Component Analysis (PCA) solves the covariance matrix $C=XX^{\top}$, where size(X)=10000X400 in our example.
You would end up with a 10000x10000 matrix, rougly 0.8GB.
Solving this  problem is not feasible, so we will need to apply a trick.
From your linear algebra lessons you know that a MxN matrix with M>N can only have N-1 non-zero eigenvalues.
So it is possible to take the eigenvalue decompostion $S=X^{\top}X$ of size NxN instead: -->

In [ ]:
def LDA(X, y, num_comp=0):
    """線形判別分析 LDA: Linear Discriminant Analysis
        X: データ行列
        y: X の各行に対応する群ラベルを格納した numpy.array リスト
    """
    #y = np.asarray(y)
    [n,d] = X.shape         # データ行列の 行:n と 列:d を求める
    n_class = np.unique(y)  # データに含まれる群の数
    if (num_comp <= 0) or (num_comp > (len(n_class)-1)):
        num_comp = (len(n_class)-1)

    Total_mean = X.mean(axis=0)             # 全平均の計算
    Sw = np.zeros((d,d), dtype=np.float32)  # 群内分散保存用
    Sb = np.zeros((d,d), dtype=np.float32)  # 群間分散保存用
    for i in n_class:                       # 各群について繰り返す
        Xi = X[np.where(y==i)[0],:]
        G_mean = Xi.mean(axis=0)                                             # 群平均
        Sw = Sw + np.dot((Xi - G_mean).T, (Xi - G_mean))                     # 群内分散
        Sb = Sb + n * np.dot((G_mean - Total_mean).T, (G_mean - Total_mean)) # 群間分散
    Eigenvalues, Eigenvectors = np.linalg.eig(np.linalg.inv(Sw) * Sb)
    idx = np.argsort(-Eigenvalues.real)
    Eigenvalues, Eigenvectors = Eigenvalues[idx], Eigenvectors[:,idx]
    Eigenvalues = np.array(Eigenvalues[0:num_comp].real, dtype=np.float32, copy=True)
    Eigenvectors = np.array(Eigenvectors[0:,0:num_comp].real, dtype=np.float32, copy=True)
    return Eigenvalues, Eigenvectors


def Fisherfaces(X,y,num_comp=0):
    #y = np.asarray(y)
    [n,d] = X.shape
    n_class = len(np.unique(y))
    #[Eigenvalues_pca, Eigenvectors_pca, mu_pca] = pca(X, y, (n-n_class))
    Eigenvalues_pca, Eigenvectors_pca, mu_pca = _PCA.Eigenvalues, _PCA.Eigenvectors, _PCA.mean
    #[Eigenvalues_lda, Eigenvectors_lda] = LDA(project(Eigenvectors_pca, X, mu_pca), y, num_comp)
    Eigenvalues_lda, Eigenvectors_lda = LDA(_PCA.projections, y, num_comp)
    Eigenvectors = np.dot(Eigenvectors_pca, Eigenvectors_lda)
    return Eigenvalues_lda, Eigenvectors #, mu_pca

Fisher_Eignevalues, Fisher_Eigenvectors = Fisherfaces(X,y)

## フィッシャー顔固有ベクトルの視覚化

In [ ]:
# help(np.set_printoptions)
N_pca = 10
fix, axes = plt.subplots(nrows=1, ncols=N_pca, figsize=(N_pca * 1.4, 1.4))
for i in range(N_pca):
    axes[i].imshow(Fisher_Eigenvectors[:,i].reshape(64,64), cmap="gray")
    axes[i].set_title(f'第 {i+1} 固有値に対応\nする固有ベクトル', size=7)
    axes[i].axis('off')

plt.show()

In [ ]:
Fisher_projections = _PCA._X @ Fisher_Eigenvectors
Fisher_reconstruct = Fisher_projections @ Fisher_Eigenvectors.T + _PCA.mean
print(Fisher_reconstruct.shape, Fisher_projections.shape)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(Fisher_Eigenvectors.T @ Fisher_Eigenvectors, center=0, vmin=-1., vmax=1., square=True, cmap='gnuplot2')
plt.title('固有ベクトルの積を可視化')
plt.show()

print((Fisher_Eigenvectors.T @ Fisher_Eigenvectors)[:5,:5])
print((Fisher_Eigenvectors.T @ Fisher_Eigenvectors)[-10:,-10:])


In [ ]:
plt.plot(Fisher_Eignevalues[3:])
plt.title('Fisher 顔の固有値プロット')
plt.show()
print(Fisher_Eignevalues[3:])

In [ ]:
# 固有顔 (PCA) を用いて 40 名の顔認識データをもちいて 3 手法を比較
X_train, X_test, y_train, y_test=train_test_split(Fisher_projections, y, test_size=split_ratio, stratify=y, random_state=42)
#X_train, X_test, y_train, y_test=train_test_split(_PCA.projections, y_sex, test_size=split_ratio, stratify=y, random_state=42)

# 共通パラメータ
params = {'max_iter':10 ** 4,
          #'C':1e3,
          #'penalty':'l2'
         }

# 3 つのモデルの定義
svc_model = SVC(**params)
logistic_model = LogisticRegression(**params)
lda_model = LinearDiscriminantAnalysis()

# 出力する図のサイズを定義
fig, _axes = plt.subplots(ncols=3, nrows=1, figsize=(12,4), constrained_layout=True)

# 3 つのモデルをそれぞれ実行して結果を描画
for i, (model_name, model) in enumerate([('サポートベクタマシン',svc_model),
                                         ('ロジスティック回帰',logistic_model),
                                         ('線形判別分析',lda_model)]):
    model.fit(X_train, y_train)

    y_hat = model.predict(X_test)  # テストデータを使って予測を行い結果を y_hat に格納

    _axes[i].imshow(metrics.confusion_matrix(y_test,y_hat), cmap='gray')
    _axes[i].set_title(f'{model_name}:分類精度:{metrics.accuracy_score(y_test,y_hat):.2f}')
    _axes[i].axis('off')

    # print(model_name, '\n', metrics.confusion_matrix(y_test,y_hat))
    # print(model_name, '\n', metrics.classification_report(y_test,y_hat))

plt.show()

In [ ]:
Fisher_reconstruct.shape
plt.figure(figsize=(2.5,2.5))
plt.imshow(Fisher_reconstruct[-1].reshape(64,64), cmap='gray')
plt.title(f'フィッシャー顔 再構成')
plt.show()

In [ ]:
# PCA による次元圧縮の結果を表示

XX = Fisher_projections
XX = Fisher_reconstruct

fig, axes = plt.subplots(ncols=3, nrows=1, figsize=(14,4))
axes[0].scatter(XX[:,0], XX[:,1], s=1)
axes[0].set_xlabel('第 1 主成分')
axes[0].set_ylabel('第 2 主成分')
for i in range(XX.shape[0]):
    axes[0].annotate(str(y[i]), (XX[i,0], XX[i,1]))

axes[1].scatter(XX[:,1], XX[:,2], s=1)
axes[1].set_xlabel('第 2 主成分')
axes[1].set_ylabel('第 3 主成分')
for i in range(XX.shape[0]):
    axes[1].annotate(str(y[i]), (XX[i,1], XX[i,2]))

axes[2].scatter(XX[:,0], XX[:,3], s=1)
axes[2].set_xlabel('第 1 主成分')
axes[2].set_ylabel('第 4 主成分')
for i in range(XX.shape[0]):
    axes[2].annotate(str(y[i]), (XX[i,0], XX[i,3]))

plt.show()